# Обработка FASTQ - файлов.

In [1]:
from Bio import SeqIO, SeqUtils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

## Чтение из файла
Будем читать риды из fasqt файла. Причем для улучшения результата добавим в программу следующие фичи:
* Не учитывать нуклеотиды с плохим качеством (за это отвечает параметр `min_perc_qual`)
* Не учитывать риды, в которых мало хороших нуклеотидов (за это отвечает параметр `min_percentage_of_good_ac`).

Код достаточно понятен:
* читаем файл по риду, считаем матрицу качеств нуклеотидов по формуле из википедии (перевод из score в probability)): 
![](q_prob.png)
* проверяем среднее качество рида, и учитываем только хорошие (см. далее, как работает)
* если ок, то запоминаем его gc-контент и качество.
* затем считаем сренее качество для рида.

УСЕ!


In [3]:
def read_and_count_fastq(filename):
#     seq - риды
#     phred_qual - качество ридов (phred_qual[i] - качество для i-го рида из seq )
#     gc_content - gc-контент i-го рида из seq
#     avg_qual - среднее качество на i-ю букву
#     min_perc_qual - минимальное качество (в процентах), меньше которого не стоит верить
#     min_percentage_of_good_ac - минимальный процент, аминокислот с хорошим качеством, для которого можно считать, что весь рид - хороший
#
    phred_qual = [[]]
    gc_content = []
    avg_qual = []
    min_perc_qual = 0.75
    min_percentage_of_good_ac = 0.4
    jj=0
    for record in SeqIO.parse(filename, "fastq"):
        cur_seq = str(record.seq)
        cur_qual = np.array(record.letter_annotations["phred_quality"])-31
        cur_qual = 1 - (10 ** ((-cur_qual)/10))
        if jj%10000 ==0:
            print(jj, end = ' ')
        if check_read_qual(cur_qual, min_perc_qual, min_percentage_of_good_ac):
            gc_content.append(count_gc_content(cur_seq, cur_qual, min_perc_qual ))
            while len(phred_qual) < len(cur_seq):
                phred_qual.append([])
            for i in range(len(phred_qual)):
                phred_qual[i].append(cur_qual[i])
        jj+=1
    print('Finished parsing, averagening...')
    jj=0
    for q in phred_qual:
        print(jj, end = ' ')
        jj+=1
        avg_qual.append(sum(q)/len(q))
    print('Finished counting!')
    return avg_qual, c_content


Функция `check_read_qual` проверяет рид на "хорошесть" - если процент ридов, в которых качество меньше чем `min_qual`, меньше чем `min_percentage_of_good_ac`, иначе - хорош.

In [4]:
def check_read_qual(qual, min_qual = 0.8, min_percentage_of_good_ac = 0.5):
    good_qual = [(1 if q >= min_qual else 0) for q in qual]
    amount_of_good = np.sum(np.array(good_qual))
    return True if amount_of_good/len(good_qual) >= min_percentage_of_good_ac else False

функция `count_gc_content` считает gc-контент, но только для нуклеотидов с качеством больше `min_qual`

In [5]:
def count_gc_content(seq, qual, min_qual = 0.8):
    gc_amount = 0
    all_amount = 0
    for i in range(len(seq)):
#       там еще есть буквы N - их соответственно считать не надо, поэтому сначала проверю, если буква - аминокислота
        if qual[i] >= min_qual and (seq[i]=='A' or seq[i]=='T' or seq[i]=='G' or seq[i]=='C' or seq[i]=='U'):
            all_amount+=1
            if seq[i]=='G' or seq[i]=='C':
                gc_amount+=1
    return round(gc_amount/all_amount, 2)
                

Строим gc-контент и качество:

In [6]:
def draw_gc_content(gc_cont):
    gc_percentage = np.array([x/100 for x in range(101)])
    gc_content = np.array(gc_cont)
    number_of_reads = np.array([np.sum(gc_content==x/100) for x in range(101)])
    
    data = np.column_stack((gc_percentage, number_of_reads))

    fig, (ax1) = plt.subplots(
        nrows=1, ncols=1,
        figsize=(10, 10)
    )

    ax1.scatter(x=gc_percentage, y=number_of_reads, marker='o', c='r', edgecolor='b')
    ax1.set_title('Scatter: GC-content')
    ax1.set_xlabel('gc percentage')
    ax1.set_ylabel('number of reads')

    plt.show()

In [7]:
def draw_qual(qual):
    position = np.array([x for x in range(len(qual))])
    quality = np.array(qual)
    
    data = np.column_stack((position, quality))

    fig, (ax1) = plt.subplots(
        nrows=1, ncols=1,
        figsize=(10, 10)
    )

    ax1.scatter(x=position, y=quality, marker='o', c='r', edgecolor='b')
    ax1.set_title('Error probabilities across all bases')
    ax1.set_xlabel('position in read')
    ax1.set_ylabel('error probability')

    plt.show()

## Запуск и проверка
Запустим наши функции на ридах, а затем запустим тоже самое в fastQC и проверим.

In [ ]:
qual, gc_content = read_and_count_fastq('data/frag.R1.fastq')
print('Drawing...')
# qual, seq, gc_content = read_and_count_fastq('data/ttt.fastq')
draw_gc_content(gc_content)
draw_qual(qual)

0 10000 20000 30000 40000 50000 60000 70000 80000 90000 100000 110000 120000 130000 140000 150000 160000 170000 180000 190000 200000 210000 220000 230000 240000 250000 260000 270000 280000 290000 300000 310000 320000 330000 340000 350000 360000 370000 380000 390000 400000 410000 420000 430000 440000 450000 460000 470000 480000 490000 500000 510000 520000 530000 540000 550000 560000 570000 580000 590000 600000 610000 620000 630000 640000 650000 660000 670000 680000 690000 700000 710000 720000 730000 740000 750000 760000 770000 780000 790000 800000 810000 820000 830000 840000 850000 860000 870000 880000 890000 900000 910000 920000 930000 940000 950000 960000 970000 980000 990000 1000000 1010000 1020000 1030000 1040000 1050000 1060000 1070000 1080000 1090000 1100000 1110000 1120000 1130000 1140000 1150000 1160000 1170000 1180000 1190000 1200000 1210000 1220000 1230000 1240000 1250000 1260000 1270000 1280000 1290000 1300000 1310000 1320000 1330000 1340000 1350000 1360000 1370000 1380000 13

Посмотрим, что дает FastQC:
![](R1_gc.png)
![](R1_qual.png)
GC-контент получился супер. Error probability вроде тоже ок. (фасткц не рисует именно то, что я рисовал)

In [ ]:
qual, seq, gc_content = read_and_count_fastq('data/frag.R2.fastq')
# qual, seq, gc_content = read_and_count_fastq('data/ttt.fastq')
draw_gc_content(gc_content)
draw_qual(qual)

Посмотрим, что дает FastQC:
![](R2_gc.png)
![](R2_qual.png)
GC-контент получился супер. Error probability вроде тоже ок. (фасткц не рисует именно то, что я рисовал)

УРА!